### Carregamento

In [28]:
# Importações necessárias
import pandas as pd

# Função para carregar os dados
def carregar_dados(caminho_arquivo, coluna_data=None):
    """
    Carrega um arquivo CSV com dados.
    
    Parâmetros:
    - caminho_arquivo (str): Caminho do arquivo CSV.
    - coluna_data (str, opcional): Nome da coluna de datas para converter para datetime.
    
    Retorna:
    - pandas.DataFrame: DataFrame com os dados carregados.
    """
    # Carrega o CSV
    dados = pd.read_csv(caminho_arquivo)
    
    # Converte a coluna de datas para datetime com timezone, se especificada
    if coluna_data:
        dados[coluna_data] = pd.to_datetime(dados[coluna_data], utc=True)
    
    return dados

# Exemplo de uso da função com coluna de datas 'date'
caminho_arquivo = '../data/raw/hr_dashboard_data.csv'
dados = carregar_dados(caminho_arquivo)

# Exibir as primeiras linhas do DataFrame
dados.head(10)

,Name,Age,Gender,Projects Completed,Productivity (%),Satisfaction Rate (%),Feedback Score,Department,Position,Joining Date,Salary
0,Douglas Lindsey,25,Male,11,57,25,4.7,Marketing,Analyst,Jan-20,63596
1,Anthony Roberson,59,Female,19,55,76,2.8,IT,Manager,Jan-99,112540
2,Thomas Miller,30,Male,8,87,10,2.4,IT,Analyst,Jan-17,66292
3,Joshua Lewis,26,Female,1,53,4,1.4,Marketing,Intern,Jan-22,38303
4,Stephanie Bailey,43,Male,14,3,9,4.5,IT,Team Lead,Jan-05,101133
5,Jonathan King,24,Male,5,63,33,4.2,Sales,Junior Developer,Jan-21,48740
6,Kyle Figueroa,33,Female,13,41,39,2.3,Sales,Analyst,Jan-18,73502
7,Shannon Allen,23,Female,4,92,68,2.8,HR,Intern,Jan-21,39670
8,Daryl Noble,30,Female,7,32,43,2.3,Marketing,Junior Developer,Jan-19,49323
9,Tracy Figueroa,39,Female,17,10,15,1.1,HR,Team Lead,Jan-04,92915


### Tamanho base

In [29]:
def verificar_tamanho_base(df):
    """
    Verifica o tamanho da base de dados em termos de número de linhas e colunas.
    
    Parâmetros:
    - df (pandas.DataFrame): DataFrame a ser verificado.
    
    Retorna:
    - dict: Dicionário com o número de linhas e colunas.
    """
    tamanho = {
        'Número de Linhas': df.shape[0],
        'Número de Colunas': df.shape[1]
    }
    
    return tamanho

# Verificar o tamanho da base de dados
tamanho_base = verificar_tamanho_base(dados)
tamanho_base

{'Número de Linhas': 200, 'Número de Colunas': 11}

### Valores nulos

In [30]:
# Função para verificar valores ausentes
def verificar_valores_ausentes(df):
    """
    Verifica a quantidade e o percentual de valores ausentes em cada coluna de um DataFrame.
    
    Parâmetros:
    - df (pandas.DataFrame): DataFrame a ser verificado.
    
    Retorna:
    - pandas.DataFrame: DataFrame com a quantidade e o percentual de valores ausentes por coluna.
    """
    # Calcular a quantidade de valores ausentes
    valores_ausentes = df.isnull().sum()
    
    # Calcular o percentual de valores ausentes
    percentual_ausentes = (valores_ausentes / df.shape[0]) * 100
    
    # Criar um DataFrame com os resultados
    tabela_ausentes = pd.DataFrame({
        'Quantidade de Valores Ausentes': valores_ausentes,
        'Percentual de Valores Ausentes (%)': percentual_ausentes
    })
    
    # Ordenar o DataFrame pelo percentual de valores ausentes em ordem decrescente
    tabela_ausentes = tabela_ausentes[tabela_ausentes['Quantidade de Valores Ausentes'] > 0]
    tabela_ausentes = tabela_ausentes.sort_values(by='Percentual de Valores Ausentes (%)', ascending=False)
    
    return tabela_ausentes

# Verificar valores ausentes
tabela_ausentes = verificar_valores_ausentes(dados)
tabela_ausentes

,Quantidade de Valores Ausentes,Percentual de Valores Ausentes (%)


### Valores duplicados

In [31]:
def detectar_valores_duplicados(df):
    """
    Detecta e remove valores duplicados de um DataFrame.
    
    Parâmetros:
    - df (pandas.DataFrame): DataFrame a ser verificado
    
    Retorna:
    - pandas.DataFrame: DataFrame sem duplicatas
    """
    # Verifica duplicatas
    duplicatas = df.duplicated(keep=False)
    n_duplicatas = duplicatas.sum()
    
    if n_duplicatas == 0:
        print("✅ Nenhum valor duplicado foi encontrado.")
        return df
    else:
        print(f"⚠️ Foram encontradas {n_duplicatas} linhas duplicadas.")
        # Remove duplicatas mantendo a primeira ocorrência
        df_sem_duplicatas = df.drop_duplicates(keep='first')
        return df_sem_duplicatas

# Aplicar a função e atualizar o DataFrame
dados = detectar_valores_duplicados(dados)
dados.head()

✅ Nenhum valor duplicado foi encontrado.


,Name,Age,Gender,Projects Completed,Productivity (%),Satisfaction Rate (%),Feedback Score,Department,Position,Joining Date,Salary
0,Douglas Lindsey,25,Male,11,57,25,4.7,Marketing,Analyst,Jan-20,63596
1,Anthony Roberson,59,Female,19,55,76,2.8,IT,Manager,Jan-99,112540
2,Thomas Miller,30,Male,8,87,10,2.4,IT,Analyst,Jan-17,66292
3,Joshua Lewis,26,Female,1,53,4,1.4,Marketing,Intern,Jan-22,38303
4,Stephanie Bailey,43,Male,14,3,9,4.5,IT,Team Lead,Jan-05,101133


In [32]:
def remover_espacos(df):
    """
    Remove espaços em branco à esquerda e à direita das strings em todas as colunas de um DataFrame.
    
    Parâmetros:
    - df (pandas.DataFrame): DataFrame a ser processado.
    
    Retorna:
    - pandas.DataFrame: DataFrame com os espaços removidos.
    - dict: Dicionário com a quantidade de espaços removidos por coluna.
    """
    # Verificar se o DataFrame é válido
    if not isinstance(df, pd.DataFrame):
        raise ValueError("O argumento fornecido não é um DataFrame.")
    
    espacos_removidos = {}
    
    # Aplicar strip em todas as colunas de tipo object (string)
    for col in df.select_dtypes(include=['object']).columns:
        espacos_antes = df[col].apply(lambda x: len(x) - len(x.strip()) if isinstance(x, str) else 0).sum()
        df[col] = df[col].map(lambda x: x.strip() if isinstance(x, str) else x)
        espacos_depois = df[col].apply(lambda x: len(x) - len(x.strip()) if isinstance(x, str) else 0).sum()
        espacos_removidos[col] = espacos_antes - espacos_depois
    
    return df, espacos_removidos

# Exemplo de uso:
dados, espacos_removidos = remover_espacos(dados)
print("Espaços removidos por coluna:")
for coluna, quantidade in espacos_removidos.items():
    print(f"{coluna}: {quantidade} espaços removidos")

dados.head()

Espaços removidos por coluna:
Name: 0 espaços removidos
Gender: 0 espaços removidos
Department: 0 espaços removidos
Position: 0 espaços removidos
Joining Date: 0 espaços removidos


,Name,Age,Gender,Projects Completed,Productivity (%),Satisfaction Rate (%),Feedback Score,Department,Position,Joining Date,Salary
0,Douglas Lindsey,25,Male,11,57,25,4.7,Marketing,Analyst,Jan-20,63596
1,Anthony Roberson,59,Female,19,55,76,2.8,IT,Manager,Jan-99,112540
2,Thomas Miller,30,Male,8,87,10,2.4,IT,Analyst,Jan-17,66292
3,Joshua Lewis,26,Female,1,53,4,1.4,Marketing,Intern,Jan-22,38303
4,Stephanie Bailey,43,Male,14,3,9,4.5,IT,Team Lead,Jan-05,101133


### Tipos dos dados

In [33]:
# Função para verificar o tipo dos dados/colunas
def verificar_tipo_dados(df):
    """
    Verifica o tipo dos dados/colunas de um DataFrame.
    
    Parâmetros:
    - df (pandas.DataFrame): DataFrame a ser verificado.
    
    Retorna:
    - pandas.DataFrame: DataFrame com os nomes das colunas e seus respectivos tipos de dados.
    """
    # Obter os tipos de dados das colunas
    tipos_dados = df.dtypes
    
    # Criar um DataFrame com os resultados
    tabela_tipos = pd.DataFrame({
        'Coluna': tipos_dados.index,
        'Tipo de Dado': tipos_dados.values
    })
    
    return tabela_tipos

# Verificar o tipo dos dados/colunas
tabela_tipos = verificar_tipo_dados(dados)
tabela_tipos

,Coluna,Tipo de Dado
0,Name,object
1,Age,int64
2,Gender,object
3,Projects Completed,int64
4,Productivity (%),int64
5,Satisfaction Rate (%),int64
6,Feedback Score,float64
7,Department,object
8,Position,object
9,Joining Date,object


### Conversão/otimização

In [40]:
def modificar_tipo_colunas(df, colunas_tipos):
    """
    Modifica o tipo de dado de colunas específicas em um DataFrame.
    
    Parâmetros:
    - df (pandas.DataFrame): DataFrame original.
    - colunas_tipos (dict): Dicionário onde as chaves são os nomes das colunas e os valores são os tipos de dados desejados.
    
    Retorna:
    - pandas.DataFrame: DataFrame com os tipos de dados modificados.
    """
    # Verifica se as colunas informadas existem no DataFrame
    colunas_invalidas = [col for col in colunas_tipos.keys() if col not in df.columns]
    if colunas_invalidas:
        raise ValueError(f"Colunas inválidas: {colunas_invalidas}")
    
    # Modifica o tipo de dado das colunas especificadas
    for coluna, tipo in colunas_tipos.items():
        try:
            df[coluna] = df[coluna].astype(tipo)
            print(f"✅ Coluna '{coluna}' convertida para '{tipo}' com sucesso.")
        except Exception as e:
            print(f"⚠️ Erro ao converter a coluna '{coluna}' para '{tipo}': {e}")
    
    return df

# Exemplo de uso:
colunas_tipos = {
    'Age': 'int8',
    'Projects Completed': 'int8',
    'Productivity (%)': 'int8',
    'Satisfaction Rate (%)': 'int8',
    'Salary': 'int32',
    'Feedback Score': 'float32',
    'Gender': 'category',
    'Position': 'category',
    'Department': 'category',
    'Joining Date': 'datetime64[ns, UTC]'

}

dados = modificar_tipo_colunas(dados, colunas_tipos)
dados.dtypes # Verificar os tipos de dados


✅ Coluna 'Age' convertida para 'int8' com sucesso.
✅ Coluna 'Projects Completed' convertida para 'int8' com sucesso.
✅ Coluna 'Productivity (%)' convertida para 'int8' com sucesso.
✅ Coluna 'Satisfaction Rate (%)' convertida para 'int8' com sucesso.
✅ Coluna 'Salary' convertida para 'int32' com sucesso.
✅ Coluna 'Feedback Score' convertida para 'float32' com sucesso.
✅ Coluna 'Gender' convertida para 'category' com sucesso.
✅ Coluna 'Position' convertida para 'category' com sucesso.
✅ Coluna 'Department' convertida para 'category' com sucesso.
✅ Coluna 'Joining Date' convertida para 'datetime64[ns, UTC]' com sucesso.


Name                                  object
Age                                     int8
Gender                              category
Projects Completed                      int8
Productivity (%)                        int8
Satisfaction Rate (%)                   int8
Feedback Score                       float32
Department                          category
Position                            category
Joining Date             datetime64[ns, UTC]
Salary                                 int32
dtype: object

## Corrigindo formato data

In [ ]:
def converter_data(df, coluna):
    """
    Converte uma coluna de datas no formato 'MMM-YY' para o tipo datetime e formata para 'MM-YYYY'.
    
    Parâmetros:
    - df (pandas.DataFrame): DataFrame contendo a coluna de datas.
    - coluna (str): Nome da coluna de datas a ser convertida.
    
    Retorna:
    - pandas.DataFrame: DataFrame com a coluna de datas convertida e formatada.
    """
    # Verificar se a coluna existe no DataFrame
    if coluna not in df.columns:
        raise ValueError(f"A coluna '{coluna}' não existe no DataFrame.")
    
    # Converter a coluna para o formato datetime
    df[coluna] = pd.to_datetime(df[coluna], format='%b-%y')
    
    # Formatar a coluna para 'MM-YYYY'
    df[coluna] = df[coluna].dt.strftime('%m-%Y')
    
    return df

# Exemplo de uso:
dados = converter_data(dados, 'Joining Date')

In [41]:
dados.head()
dados.dtypes

Name                                  object
Age                                     int8
Gender                              category
Projects Completed                      int8
Productivity (%)                        int8
Satisfaction Rate (%)                   int8
Feedback Score                       float32
Department                          category
Position                            category
Joining Date             datetime64[ns, UTC]
Salary                                 int32
dtype: object

In [ ]:
def modificar_nomes_colunas(df, colunas_nomes):
    """
    Modifica os nomes de colunas específicas em um DataFrame.
    
    Parâmetros:
    - df (pandas.DataFrame): DataFrame original.
    - colunas_nomes (dict): Dicionário onde as chaves são os nomes atuais das colunas e os valores são os novos nomes desejados.
    
    Retorna:
    - pandas.DataFrame: DataFrame com os nomes das colunas modificados.
    """
    # Verifica se as colunas informadas existem no DataFrame
    colunas_invalidas = [col for col in colunas_nomes.keys() if col not in df.columns]
    if colunas_invalidas:
        raise ValueError(f"Colunas inválidas: {colunas_invalidas}")
    
    # Renomeia as colunas especificadas
    df = df.rename(columns=colunas_nomes)
    print(f"✅ Colunas renomeadas com sucesso: {colunas_nomes}")
    
    return df

# Exemplo de uso:
colunas_nomes = {
    'Age': 'age',
    'Projects Completed': 'projects_completed',
    'Productivity (%)': 'productivity_percent',
    'Satisfaction Rate (%)': 'satisfaction_rate_percent',
    'Salary': 'salary',
    'Feedback Score': 'feedback_score',
    'Gender': 'gender',
    'Position': 'position',
    'Department': 'department',
    'Joining Date': 'joining_date',
    'Name': 'name'
}

dados = modificar_nomes_colunas(dados, colunas_nomes)


In [49]:
dados.head() # Verificar os novos nomes das colunas

,name,age,gender,projects_completed,productivity_percent,satisfaction_rate_percent,feedback_score,department,position,joining_date,salary
0,Douglas Lindsey,25,Male,11,57,25,4.7,Marketing,Analyst,2020-01-01 00:00:00+00:00,63596
1,Anthony Roberson,59,Female,19,55,76,2.8,IT,Manager,1999-01-01 00:00:00+00:00,112540
2,Thomas Miller,30,Male,8,87,10,2.4,IT,Analyst,2017-01-01 00:00:00+00:00,66292
3,Joshua Lewis,26,Female,1,53,4,1.4,Marketing,Intern,2022-01-01 00:00:00+00:00,38303
4,Stephanie Bailey,43,Male,14,3,9,4.5,IT,Team Lead,2005-01-01 00:00:00+00:00,101133


In [52]:
import re

def verificar_formato_coluna(df, coluna, tipo):
    """
    Verifica o formato dos dados em uma coluna específica de um DataFrame com base no tipo especificado.
    
    Parâmetros:
    - df (pandas.DataFrame): DataFrame contendo a coluna a ser verificada.
    - coluna (str): Nome da coluna a ser verificada.
    - tipo (str): Tipo de formato a ser verificado. Pode ser 'numero', 'virgula', 'title_case', 'primeira_maiuscula', etc.
    
    Retorna:
    - pandas.DataFrame: DataFrame com as linhas que não correspondem ao formato especificado.
    """
    if coluna not in df.columns:
        raise ValueError(f"A coluna '{coluna}' não existe no DataFrame.")
    
    padroes = {
        'numero': r'^\d+$',
        'virgula': r'^[^,]+(,[^,]+)*$',
        'title_case': r'^[A-Z][a-z]+(?: [A-Z][a-z]+)*$',
        'primeira_maiuscula': r'^[A-Z][a-z]*$'
    }
    
    if tipo not in padroes:
        raise ValueError(f"Tipo '{tipo}' não é suportado. Tipos suportados: {list(padroes.keys())}")
    
    padrao = padroes[tipo]
    mascara = df[coluna].apply(lambda x: bool(re.match(padrao, str(x))))
    linhas_invalidas = df[~mascara]
    
    return linhas_invalidas

# Exemplo de uso:
coluna = 'name'
tipo = 'title_case'  # Verificar se os nomes estão em 'title_case'
linhas_invalidas = verificar_formato_coluna(dados, coluna, tipo)
print(f"Linhas que não correspondem ao tipo '{tipo}':")
linhas_invalidas


Linhas que não correspondem ao tipo 'title_case':


,name,age,gender,projects_completed,productivity_percent,satisfaction_rate_percent,feedback_score,department,position,joining_date,salary
13,Ms. Emily Dawson,32,Female,14,45,20,3.1,Marketing,Analyst,2018-01-01 00:00:00+00:00,70271
19,Michael Morton DDS,23,Male,2,1,17,4.4,Sales,Intern,2021-01-01 00:00:00+00:00,37855
53,Alexis Smith DDS,44,Male,16,60,80,3.2,HR,Senior Developer,2007-01-01 00:00:00+00:00,80976
62,Dr. Corey Orr,28,Female,5,13,40,4.1,IT,Analyst,2020-01-01 00:00:00+00:00,70439
86,Carrie Lowe DVM,23,Female,5,9,50,3.5,HR,Intern,2020-01-01 00:00:00+00:00,39138
118,Mr. Brandon Weber,31,Female,6,37,90,1.9,IT,Analyst,2017-01-01 00:00:00+00:00,60575
146,Mrs. Heidi Villa,23,Male,1,11,7,1.6,Marketing,Intern,2022-01-01 00:00:00+00:00,34697
171,Mrs. Laura Gonzalez,25,Male,4,86,94,2.9,Marketing,Intern,2021-01-01 00:00:00+00:00,34601
182,Dr. Shane Lewis,28,Female,18,34,5,2.3,Sales,Senior Developer,2015-01-01 00:00:00+00:00,86941


### Carga

In [50]:
def salvar_dados(df, caminho_arquivo_saida):
    """
    Salva o DataFrame em um arquivo CSV.
    
    Parâmetros:
    - df (pandas.DataFrame): DataFrame a ser salvo.
    - caminho_arquivo_saida (str): Caminho do arquivo CSV de saída.
    """
    df.to_csv(caminho_arquivo_saida, index=False)
    print(f"✅ Dados salvos com sucesso em '{caminho_arquivo_saida}'")

# Exemplo de uso:
caminho_arquivo_saida = '../data/processed/hr_dashboard_data_atualizado.csv'
salvar_dados(dados, caminho_arquivo_saida)

✅ Dados salvos com sucesso em '../data/processed/hr_dashboard_data_atualizado.csv'
